# Predicción de comportamientos con Word2vec

## Importando Datos

### Extrayendo la secuencia de Comportamientos Colectivos de los Clientes

In [1]:
import pandas as pd
import gensim
from gensim import corpora, models, similarities

In [2]:
# data = pd.read_csv("../resultados/mobile_consume1_results.csv")
data = pd.read_csv("../../resultados/mobile_consume1_results.csv")

In [3]:
# Lista de Clientes
clientes = data.CO_ID.unique()

In [4]:
# Lista de Periodos para la Opcion 1
year_min = min(data.YEAR) 
year_max = max(data.YEAR) 
week_min = min(data.WEEK) 
week_max = max(data.WEEK) 

periodos = []

if year_min == year_max:
    year = year_min
    for week in range(week_min,week_max+1):
        weeks = ""
        if week < 10:
            weeks = "0"+str(week)
        else:
            weeks = str(week)
        periodos.append(str(year)+"-"+weeks)
        
else:
    for year in range(year_min,year_max+1):
        if year == year_min:
            for week in range(week_min,53):
                
                weeks = ""
                if week < 10:
                    weeks = "0"+str(week)
                else:
                    weeks = str(week)
                periodos.append(str(year)+"-"+weeks)
                #print(year,weeks)
                
        elif year == year_max:   
            for week in range(1,week_max+1):
                
                weeks = ""
                if week < 10:
                    weeks = "0"+str(week)
                else:
                    weeks = str(week)
                periodos.append(str(year)+"-"+weeks)
                #print(year,weeks)
                
        else:    
            for week in range(1,53):
            
                weeks = ""
                if week < 10:
                    weeks = "0"+str(week)
                else:
                    weeks = str(week)
                periodos.append(str(year)+"-"+weeks)
                #print(year,weeks)


## Construyendo CORPUS 

### Opción 1 (considerando "SinActividad")

In [5]:
# Construyendo CORPUS CON CADA TEMPORADA DE CLIENTES (considerando las temporadas vacias)

comportamientos=[]
    
for index, cliente in enumerate(clientes):
    lista = []
    for index, periodo in enumerate(periodos):
        lista.append("SinActividad")
        
    comportamientos.append(lista)
    

In [6]:
lista_periodos = list(periodos)

for i in range(len(data)):
    # en la comportamiendo del cliente i
    fila = data.loc[i,['CO_ID','YEAR','WEEK','COLLECTIVE_CLUSTER']]
    # en el indice del cliente i_c  y del periodo i_p 
    index_c = list(clientes).index(int(fila[0]))
    week = ""
    if int(fila[2]) < 10:
        week = "0" + str(int(fila[2]))
    else:
        week = str(int(fila[2]))
    tem = str(int(fila[1]))+"-"+week
    index_p = lista_periodos.index(tem)
        
    try:
        num = ""
        if int(fila[3]) < 10:
            num = "0" + str(int(fila[3]))
        else:
            num = str(int(fila[3]))
        
        comportamientos[index_c][index_p] = "cluster_"+num
    except Exception as e:
        print("Oops!  That was no valid index.  Try again...")
    

In [7]:
tok_corp = comportamientos

### Opción 2 (sin considerar "SinActividad")

In [ ]:
tok_corp = []

for cli in clientes:
    data_i = data[(data.CO_ID == cli)]
    lista_i = list(data_i.COLLECTIVE_CLUSTER)

    for j in range(len(lista_i)):
        dat = ""
        if int(str(lista_i[j])) < 10:
            dat = "0" + str(lista_i[j])
        else:
            dat = str(int(fila[2]))
            
        lista_i[j] = "cluster_"+dat
    tok_corp.append(lista_i)
    

In [ ]:
tok_corp

## Modelo

In [66]:
model = gensim.models.Word2Vec(tok_corp, min_count = 5, window=5, size = 7)

In [70]:
# Diccionario

# sorted(model.wv.index2word)

In [68]:
# VOCABULARIO

print(sorted(list(model.wv.vocab.keys())))
print("Cantidad = "+str(len(model.wv.vocab)))

['SinActividad', 'cluster_00', 'cluster_01', 'cluster_02', 'cluster_03', 'cluster_04', 'cluster_05', 'cluster_06', 'cluster_07', 'cluster_08', 'cluster_09', 'cluster_10', 'cluster_11', 'cluster_12', 'cluster_13', 'cluster_14', 'cluster_15', 'cluster_16', 'cluster_17', 'cluster_18', 'cluster_19', 'cluster_20', 'cluster_21', 'cluster_22', 'cluster_23', 'cluster_24', 'cluster_25', 'cluster_26', 'cluster_27', 'cluster_28', 'cluster_29', 'cluster_30', 'cluster_31', 'cluster_32', 'cluster_33', 'cluster_34', 'cluster_35', 'cluster_36', 'cluster_37', 'cluster_38', 'cluster_39', 'cluster_40', 'cluster_41', 'cluster_42', 'cluster_43', 'cluster_44', 'cluster_45', 'cluster_46', 'cluster_47', 'cluster_48', 'cluster_49', 'cluster_50', 'cluster_51', 'cluster_52', 'cluster_53', 'cluster_54', 'cluster_55', 'cluster_56', 'cluster_57', 'cluster_58', 'cluster_59', 'cluster_60', 'cluster_61', 'cluster_62', 'cluster_63', 'cluster_64', 'cluster_65', 'cluster_66', 'cluster_67', 'cluster_68', 'cluster_69']
Can

### Resultados

In [27]:
model.most_similar("cluster_00")

/home/jkn/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
/home/jkn/anaconda3/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('cluster_27', 0.8433127403259277),
 ('cluster_41', 0.8374015092849731),
 ('cluster_09', 0.8179642558097839),
 ('cluster_44', 0.815779447555542),
 ('cluster_32', 0.7600679397583008),
 ('cluster_35', 0.7594320178031921),
 ('cluster_04', 0.7386809587478638),
 ('cluster_59', 0.7298378348350525),
 ('cluster_47', 0.7173379063606262),
 ('cluster_39', 0.707004189491272)]

In [28]:
import random

In [63]:
prueba = []

"""for i in range(3): 
    prueba.append("cluster_"+str(random.randint(0,69))+".0")
    """
prueba = ['cluster_29',"cluster_00", "SinActividad", "SinActividad"]

In [64]:
prueba

['cluster_29', 'cluster_00', 'SinActividad', 'SinActividad']

In [65]:
# Predicion de la siguiente palabra

model.predict_output_word(prueba, topn=5)

[('cluster_00', 0.03985915),
 ('cluster_27', 0.03670628),
 ('cluster_41', 0.03650439),
 ('cluster_09', 0.03147976),
 ('cluster_35', 0.028495274)]

In [56]:
model.predict_output_word(prueba, topn=20)

[('cluster_49', 0.02102846),
 ('cluster_35', 0.020178417),
 ('cluster_15', 0.020018263),
 ('cluster_07', 0.019643178),
 ('cluster_40', 0.019266814),
 ('cluster_25', 0.019191433),
 ('cluster_27', 0.019084098),
 ('cluster_66', 0.018588684),
 ('cluster_56', 0.018498663),
 ('cluster_46', 0.018433262),
 ('cluster_04', 0.017946504),
 ('cluster_26', 0.017863272),
 ('cluster_44', 0.01784515),
 ('cluster_32', 0.01722876),
 ('cluster_55', 0.017153617),
 ('cluster_64', 0.016960876),
 ('cluster_59', 0.016897112),
 ('cluster_47', 0.016843623),
 ('cluster_36', 0.01680604),
 ('cluster_60', 0.016775558)]

In [46]:
# Modelos

model["cluster_00"]

/home/jkn/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  This is separate from the ipykernel package so we can avoid doing imports until


array([-4.012762  ,  0.37017253, -3.2437015 ,  0.18191081, -1.0842882 ,
        1.6514736 , -0.7659784 ], dtype=float32)

In [47]:
model["SinActividad"]

/home/jkn/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


array([ 0.13033447,  0.2978028 , -0.72173613,  0.21796066, -0.3534182 ,
        0.6917916 ,  0.27593198], dtype=float32)

In [48]:
# Parecidos

model.most_similar(["SinActividad"])

/home/jkn/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  This is separate from the ipykernel package so we can avoid doing imports until
/home/jkn/anaconda3/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('cluster_05', 0.6292538642883301),
 ('cluster_31', 0.6106607913970947),
 ('cluster_51', 0.5814326405525208),
 ('cluster_12', 0.5633707642555237),
 ('cluster_45', 0.5283991098403931),
 ('cluster_24', 0.5118153095245361),
 ('cluster_49', 0.5116891860961914),
 ('cluster_59', 0.5043812990188599),
 ('cluster_00', 0.5043807029724121),
 ('cluster_09', 0.4900956153869629)]

In [49]:
model.most_similar(["cluster_05"])

/home/jkn/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
/home/jkn/anaconda3/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('cluster_51', 0.9146233201026917),
 ('cluster_12', 0.8005421757698059),
 ('cluster_62', 0.7763609886169434),
 ('cluster_50', 0.7576356530189514),
 ('cluster_48', 0.7394227981567383),
 ('cluster_08', 0.7371989488601685),
 ('cluster_37', 0.7224788069725037),
 ('cluster_40', 0.7164984345436096),
 ('cluster_41', 0.7058554887771606),
 ('cluster_59', 0.6882692575454712)]

In [50]:
model.most_similar(["cluster_51"])

/home/jkn/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
/home/jkn/anaconda3/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('cluster_05', 0.9146233201026917),
 ('cluster_08', 0.7733933925628662),
 ('cluster_62', 0.7488337755203247),
 ('cluster_40', 0.7364148497581482),
 ('cluster_50', 0.7140761613845825),
 ('cluster_57', 0.7080239057540894),
 ('cluster_07', 0.6977671384811401),
 ('cluster_49', 0.668401837348938),
 ('cluster_12', 0.6642203330993652),
 ('cluster_37', 0.6386265158653259)]

### Referencias

In [51]:

# https://github.com/RaRe-Technologies/gensim/tree/develop/docs/notebooks
    